# 🗄️ GDPR Compliance Agent - Notebook 2: Pinecone Vector Database Setup

## 📋 Table of Contents
1. [Overview](#overview)
2. [Setup & Environment](#setup--environment)
3. [Load Processed Data](#load-processed-data)
4. [Initialize OpenAI Embeddings](#initialize-openai-embeddings)
5. [Pinecone Setup](#pinecone-setup)
6. [Upload to Vector Database](#upload-to-vector-database)
7. [Verification & Testing](#verification--testing)
8. [Next Steps](#next-steps)

---

## 🎯 Overview

**Goal**: Upload our processed GDPR document chunks to Pinecone vector database and create embeddings using OpenAI.

**This Notebook Focus**:
- Load chunks processed in Notebook 1
- Initialize OpenAI embeddings
- Connect to Pinecone vector database
- Upload documents with embeddings
- Verify the setup works

**Key Technologies**:
- OpenAI `text-embedding-3-small` for embeddings
- Pinecone for vector storage and search
- LangChain for orchestration



---

## ⚙️ Setup & Environment

*Import required libraries and set up environment variables*

In [ ]:
# Cell 1: Setup and Imports
import os
import sys
import pickle
import time
from dotenv import load_dotenv

# Add project root to Python path
sys.path.append(os.path.abspath('..'))

# Load environment variables
load_dotenv()

# LangChain imports
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Pinecone imports
import pinecone
from pinecone import ServerlessSpec

# Helper functions
from src.embedding_cost_calculator import calculate_embedding_cost


/Users/guillermo/venvs/langchain_venv/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [3]:
# Cell 2: Load Environment Variables
# Load API keys from .env file
load_dotenv()

# Verify environment variables
print("🔑 Environment Configuration:")
print(f"   OpenAI API Key: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")
print(f"   Pinecone API Key: {'✅' if os.getenv('PINECONE_API_KEY') else '❌'}")
# print(f"   Pinecone Environment: {'✅' if os.getenv('PINECONE_ENVIRONMENT') else '❌'}")

# if not all([os.getenv('OPENAI_API_KEY'), os.getenv('PINECONE_API_KEY'), os.getenv('PINECONE_ENVIRONMENT')]):
if not all([os.getenv('OPENAI_API_KEY'), os.getenv('PINECONE_API_KEY')]):
    print("\n⚠️  Missing environment variables!")
    print("   Please check your .env file contains:")
    print("   - OPENAI_API_KEY")
    print("   - PINECONE_API_KEY")
    # print("   - PINECONE_ENVIRONMENT")
    print("\n   These are required for this notebook.")

🔑 Environment Configuration:
   OpenAI API Key: ✅
   Pinecone API Key: ✅


---

## 📥 Load Processed Data

*Load the chunks and configuration saved from Notebook 1*


In [4]:
# Cell 3: Load Processed Chunks
def load_processed_data():
    """Load chunks and config from Notebook 1 processing"""
    try:
        # Load configuration
        with open("../2_data/processed/config.pkl", "rb") as f:
            config = pickle.load(f)
        
        # Load chunks
        with open("../2_data/processed/chunks.pkl", "rb") as f:
            serializable_chunks = pickle.load(f)
        
        # Recreate Document objects
        chunks = []
        for chunk_data in serializable_chunks:
            doc = Document(
                page_content=chunk_data['page_content'],
                metadata=chunk_data['metadata']
            )
            chunks.append(doc)
        
        print("✅ Successfully loaded processed data:")
        print(f"   - Chunks: {len(chunks)}")
        print(f"   - Chunk size: {config['chunk_size']}")
        print(f"   - Overlap: {config['chunk_overlap']}")
        print(f"   - Estimated tokens: {config['total_tokens']:,}")
        print(f"   - Estimated cost: ${config['estimated_cost']:.4f}")
        
        return chunks, config
        
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
        print("💡 Please run Notebook 1 first to process the PDF")
        return [], {}
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return [], {}

# Load the data
chunks, config = load_processed_data()

if not chunks:
    print("❌ Cannot proceed without processed chunks.")
else:
    # Show sample chunk
    print(f"\n📋 Sample chunk metadata:")
    print(chunks[0].metadata)
    print(f"\n📝 Sample content preview:")
    print(chunks[0].page_content[:200] + "...")

✅ Successfully loaded processed data:
   - Chunks: 266
   - Chunk size: 800
   - Overlap: 120
   - Estimated tokens: 46,778
   - Estimated cost: $0.0009

📋 Sample chunk metadata:
{'document_type': 'zdh_gdpr_handbook', 'document_name': 'ZDH_LEITFADEN_DATENSCHUTZ_BETRIEBE_HANDWERKER.pdf', 'language': 'german', 'source': '../2_data/raw/ZDH_LEITFADEN_DATENSCHUTZ_BETRIEBE_HANDWERKER.pdf', 'page_number': 1, 'total_pages': 99, 'content_length': 121, 'content_category': 'legal_basis', 'section_type': 'content', 'creationdate': '2020-11-06T11:24:59+01:00', 'author': 'Kasper, Lisa', 'moddate': '2020-11-06T11:24:59+01:00', 'page': 0, 'page_label': '1', 'chunk_id': 1, 'chunk_size': 121, 'total_chunks': 266}

📝 Sample content preview:
Leitfaden 
Datenschutzrecht 
Was Betriebe zu beachten haben 
 
 
Stand: November 2020 
 
Abteilung Organisation und Recht...



---

## 🤖 Initialize OpenAI Embeddings

*Set up the embedding model that will convert text to vectors*


In [5]:
# Cell 4: Initialize OpenAI Embeddings
def initialize_embeddings():
    """Initialize OpenAI embeddings without test query"""
    try:
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=os.getenv('OPENAI_API_KEY')
        )
        
        print("✅ OpenAI Embeddings initialized successfully!")
        print(f"   Model: text-embedding-3-small")
        print(f"   Dimension: 1536 (for text-embedding-3-small)")
        
        return embeddings
        
    except Exception as e:
        print(f"❌ Error initializing OpenAI embeddings: {e}")
        print("💡 Check your OPENAI_API_KEY in .env file")
        return None

# Initialize embeddings
embeddings = initialize_embeddings()

✅ OpenAI Embeddings initialized successfully!
   Model: text-embedding-3-small
   Dimension: 1536 (for text-embedding-3-small)


---

## 🗃️ Pinecone Setup

*Initialize connection to Pinecone vector database*

In [6]:
# Cell 5: Pinecone Setup (Optimized for PineconeVectorStore)
def initialize_pinecone_for_vectorstore(index_name="gdpr-compliance-openai"):
    """Initialize Pinecone specifically for PineconeVectorStore compatibility"""
    try:
        from pinecone import Pinecone, ServerlessSpec
        
        pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
        
        print("✅ Pinecone initialized for PineconeVectorStore!")
        
        existing_indexes = pc.list_indexes().names()
        print(f"📋 Existing indexes: {existing_indexes}")
        
        if index_name not in existing_indexes:
            print(f"📦 Creating new index: {index_name}")
            
            pc.create_index(
                name=index_name,
                dimension=1536,
                metric='cosine',
                spec=ServerlessSpec(
                    cloud='aws',
                    region='us-east-1'
                )
            )
            
            print("⏳ Waiting for index to initialize...")
            while not pc.describe_index(index_name).status['ready']:
                time.sleep(1)
            
            print("✅ Index created and ready!")
        else:
            print(f"✅ Using existing index: {index_name}")
        
        # Return both pc and index for flexibility
        index = pc.Index(index_name)
        time.sleep(1)
        
        stats = index.describe_index_stats()
        print(f"📊 Index statistics:")
        print(f"   - Total vectors: {stats['total_vector_count']}")
        
        return pc, index
        
    except Exception as e:
        print(f"❌ Error initializing Pinecone: {e}")
        return None, None

# Initialize Pinecone
pc, index = initialize_pinecone_for_vectorstore()

✅ Pinecone initialized for PineconeVectorStore!
📋 Existing indexes: ['extractive-question-answering', 'gdpr-compliance-openai', 'langchain-retrieval-agent', 'abstractive-qa-history', 'gdpr-compliance']
✅ Using existing index: gdpr-compliance-openai
📊 Index statistics:
   - Total vectors: 0




---

## 🚀 Upload to Vector Database

*Upload document chunks to Pinecone with embeddings*



In [9]:
# # Cell 6: Current Vector Store Setup

def create_pinecone_vectorstore_simple(chunks, index_name="gdpr-compliance-openai"):
    """Simple version without cost calculation"""
    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    print("🔄 Creating Pinecone vector store...")
    vectorstore = PineconeVectorStore.from_documents(
        documents=chunks,
        embedding=embeddings,
        index_name=index_name
    )
    
    print(f"✅ Successfully loaded {len(chunks)} documents into Pinecone")
    
    # Optional: Show basic stats
    total_chars = sum(len(chunk.page_content) for chunk in chunks)
    print(f"📊 Stats: {len(chunks)} chunks, ~{total_chars} characters")
    
    return vectorstore

# Use the simple version
vectorstore = create_pinecone_vectorstore_simple(chunks)


🔄 Creating Pinecone vector store...
✅ Successfully loaded 266 documents into Pinecone
📊 Stats: 266 chunks, ~170805 characters



---

## ✅ Verification & Testing

*Verify that the upload worked and test search functionality*. (No Tokens used, only search similarity?)


In [14]:
# Cell 5: Test Vector Store and Build RAG Chain
print("🧪 Testing Vector Store Retrieval...")

# Test with some sample queries
# Test with German queries that match your data protection content
test_queries = [
    "Was ist die Datenschutzrichtlinie?",
    "Wie sollen Kundendaten behandelt werden?",
    "Was sind die GDPR-Anforderungen?",
    "Wie geht man mit personenbezogenen Daten um?",
    "Was muss bei Datenverarbeitung beachtet werden?",
    "Welche Rechte haben Kunden bezüglich ihrer Daten?"
]


for query in test_queries:
    print(f"\n🔍 Query: '{query}'")
    results = vectorstore.similarity_search(query, k=2)
    
    print(f"   Found {len(results)} relevant chunks:")
    for i, doc in enumerate(results):
        print(f"   {i+1}. {doc.page_content[:150]}...")
    print("   " + "─" * 50)

print("\n✅ Vector store test completed!")

🧪 Testing Vector Store Retrieval...

🔍 Query: 'Was ist die Datenschutzrichtlinie?'
   Found 2 relevant chunks:
   1. Leitfaden 
Datenschutzrecht 
Was Betriebe zu beachten haben 
 
 
Stand: November 2020 
 
Abteilung Organisation und Recht...
   2. 3. Formelle Pflichten von Betrieben – Ein Überblick  
 
Welchen Zweck verfolgen die Pflichten?  
 
Das Datenschutzrecht räumt Personen, deren Daten vo...
   ──────────────────────────────────────────────────

🔍 Query: 'Wie sollen Kundendaten behandelt werden?'
   Found 2 relevant chunks:
   1. Kunde 
 
 
Familienname 
 
 
Vorname 
 
 
Geburtsname 
 
 
Geschlecht 
 
 
Geburtsdatum 
 
 
Staatsangehörigkeit 
 
 
Straße 
 
 
PLZ 
 
 
Wohnort 
 
...
   2. Gesetz vorgesehene Informationen zu erteilen. Dies sind im Einzelnen: 
  
◼ Alle über den Betroffenen gespeicherten Daten (z.B. Name, Anschrift, E -Ma...
   ──────────────────────────────────────────────────

🔍 Query: 'Was sind die GDPR-Anforderungen?'
   Found 2 relevant chunks:
   1. Rechtliche

------
------